## For Streamlit Dashboard

Adjusted to new route_tankerkoenig_integration.py script

In [ ]:
streamlit_code = '''
import streamlit as st
import pandas as pd
import joblib
from src.integration.route_tankerkoenig_integration import get_fuel_prices_for_route

# Load models once at startup
@st.cache_resource
def load_models():
    return {
        "e5": joblib.load("models/fuel_price_model_e5_v1.joblib"),
        "e10": joblib.load("models/fuel_price_model_e10_v1.joblib"),
        "diesel": joblib.load("models/fuel_price_model_diesel_v1.joblib")
    }

models = load_models()

st.title("⛽ Fuel Price Optimizer")

# User inputs
col1, col2 = st.columns(2)
with col1:
    start_locality = st.text_input("From (city):", "Tübingen")
    start_address = st.text_input("Start address:", "Wilhelmstraße 7")

with col2:
    end_locality = st.text_input("To (city):", "Reutlingen")
    end_address = st.text_input("End address:", "Charlottenstraße 45")

fuel_type = st.selectbox("Fuel type:", ["e5", "e10", "diesel"])

if st.button("🔍 Find Cheapest Station"):
    with st.spinner("Calculating route and predicting prices..."):
        try:
            # Get data from integration
            model_input = get_fuel_prices_for_route(
                start_locality=start_locality,
                end_locality=end_locality,
                start_address=start_address,
                end_address=end_address,
                use_realtime=True
            )
            
            if not model_input:
                st.error("No stations found along route.")
            else:
                # Convert to DataFrame
                df = pd.DataFrame(model_input)
                
                # Prepare features for model
                suffix = f"_{fuel_type}"
                model_features = df[[
                    f'price_lag_1d{suffix}',
                    f'price_lag_2d{suffix}',
                    f'price_lag_3d{suffix}',
                    f'price_lag_7d{suffix}'
                ]].copy()
                
                model_features.columns = [
                    'price_lag_1d', 'price_lag_2d',
                    'price_lag_3d', 'price_lag_7d'
                ]
                
                # Predict
                predictions = models[fuel_type].predict(model_features)
                df['predicted_price'] = predictions
                
                # Find cheapest
                cheapest_idx = predictions.argmin()
                cheapest = df.iloc[cheapest_idx]
                
                # Display result
                st.success(f"💰 Cheapest: {cheapest['tk_name']} - €{cheapest['predicted_price']:.3f}/L")
                
                # Show all stations
                st.subheader("All Stations (Sorted by Price)")
                summary = df[[
                    'tk_name', 'brand', 'city',
                    'distance_along_m', 'predicted_price'
                ]].sort_values('predicted_price')
                
                st.dataframe(summary)
                
        except Exception as e:
            st.error(f"Error: {e}")
'''

print("Streamlit Dashboard Code:")
print(streamlit_code)